In [1]:
import importlib
import os
import sys
import pandas as pd

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(parent_dir)

import shared.data
import shared.data.loaders.gp
import shared.data.loaders.wsc

In [2]:
importlib.reload(shared.data)
importlib.reload(shared.data.loaders.gp)
importlib.reload(shared.data.loaders.wsc)

wsc = shared.data.loaders.wsc.load_wsc(csv_directory="../data/raw/wsc")
combined_df = shared.data.loaders.gp.load_gp(csv_directory="../data/processed/gp", output_csv=None)

flattened = shared.data.create_flat_dataset(combined_df, metric="points")

In [ ]:
def attemped_mapping(wsc_df, gp_df):
    wsc_mapped = wsc_df.copy()
    gp_index = gp_df[["Name", "Country", "Nick", "user_pseudo_id"]].copy()
    gp_index["name_lc"] = gp_index["Name"].str.lower()

    wsc_mapped["Name"] = wsc_mapped["Name"].str.replace(",", "")
    wsc_mapped["Name"] = wsc_mapped["Name"].str.title()
    wsc_mapped["name_lc"] = wsc_mapped["Name"].str.lower()

    parts = wsc_mapped["Name"].str.split()
    wsc_mapped["flipped_name"] = parts.apply(lambda x: " ".join(reversed(x)) if isinstance(x, list) else "")

    joined = pd.merge(wsc_mapped, gp_index, on="Name", how="left")
    joined_flipped = pd.merge(joined, gp_index, left_on="flipped_name", right_on="Name", how="left")
    wsc_and_gp = pd.merge(joined_flipped, gp_index, on="name_lc", how="left")

    # Otherwise the "Name" column comes from `gp_index` in the last join
    wsc_and_gp.drop(columns=["Name"], inplace=True)
    wsc_and_gp.rename(columns={"Name_x": "Name"}, inplace=True)
    
    # Use our attempted identifiers in a hierarchy by name, then flipped name, then lowercase name
    wsc_and_gp["matched_id"] = wsc_and_gp["user_pseudo_id_x"].combine_first(
        wsc_and_gp["user_pseudo_id_y"]).combine_first(wsc_and_gp["user_pseudo_id"])

    manual_map = {
           "Zvěřina Jan": "Jan Zverina (Nickless) Czech Rep.",
    "Demiger Matúš": "Matus Demiger (Nickless) Slovakia",
    "De Laat Bram": "Bram de Laat (Para) Netherlands",
    "Bram De Laat": "Bram de Laat (Para) Netherlands",
    "Hudák Peter": "Peter Hudak (ppeetteerr) Slovakia",
    "Beenhakker Chiel": "Chiel Beenhakker Beenhakker (bakpao) - Netherlands",
    "Hu Yuxuan": "Hu Yuxuan Hu (huyuxuan) - China",
    "Qiu Suzhe": "Suzhe QIU (ysbg123) - China",
    "Yang Leduo": "Le Duo Yang (Sky Yang) - China",
    "Xie Yifan": "YiFan Xie (evan_xie) - China",
    "Prasanna Venkatesh Seshadri": "Prasanna Seshadri (prasanna16391) - India",
    "Yu Tianxiang": "TianXiang Yu Yu (Yutx) - China",
    "Kishore Kumar Sridharan": "Kishore Sridharan (kishy72) - India",
    "Huang Xiaowei": "Xiao Wei Huang (xiao01wei) - China",
    "Highryll Cj Tan": "CJ Tan (hetan) - Philippines",
    "Neil Gary Zussman": "Neil Zussman (Nilz) - UK",
    "Kartik Reddy Mogiligundla": "Kartik Reddy (Nickless) - India",
    "Zoltán Horváth": "Zoltan Horvath (Valezius) - Hungary",
    "Thomas Collyer": "Tom Collyer (Nickless) - UK",
    "Hong Seonghwa": "SeongHwa Hong (Nickless) - Korea, South",
    "Guenther Susanne": "Susanne Günther (zuzanina) - Germany",
    "Schuckert Eva-Maria": "Eva Schuckert (Nickless) - Austria",
    "Jaipal Reddy Mogiligundla": "Jaipal Reddy (mjaipal) - India",
    "Olga Diaz Moruno": "Olga Diaz (Nickless) - Spain",
    "Özgün Onur Özdemir": "Özgün Onur ÖZDEMİR (Nickless) - Turkey",
    "Terveer Sara Karina": "Sara Terveer (Nickless) - Germany",
    "Roger Peter Kohler": "Roger Kohler (ropeko) - Switzerland",
    "Plassmann Volker Conrad": "Volker Plassmann (Nickless) - Germany",
    "Jin Sungwon": "Sung-Won Jin (swj219) - Korea, South",
    "David Mcneill": "David McNeill (Nickless) - UK",
    "Keisu Okuma": "Keisui Okuma (Nickless) - Japan",
    "Schukert Eva": "Eva Schuckert (Nickless) - Austria",
    "Piibeleht Tiit Hendrik": "Tiit Hendrik Piibeleht (Tiiduke) - Estonia",
    "Vesna Jovanović": "Vesna Jovanovic (vesnasta) - Serbia",
    "Michaël Genier": "Michael Genier (LoHoX) - Switzerland",
    "Poulsen Henning Kalsgaard": "Henning Kalsgaard Poulsen (Kalsgaard) - Denmark",
    "Buyukkale Yunus Emre": "Yunus Emre Büyükkale (ynus) - Turkey",
    "Mladen Meštrović": "Mladen Mestrovic (mestar) - Croatia",
    "Brütsch Christof": "Christof Bruetsch (chrishy) - Switzerland",
    "Grigoļunovičs Andrejs": "Andrew Grigolunovich (AGCFA) - Latvia",
    "Jeļena Balanova": "Jelena Balanova (Nickless) - Latvia",
    'Alberto Filippini': "alberto filippini (albfilip) - Italy",
    "Filippini Alberto": "alberto filippini (albfilip) - Italy",
    'Sara Martin': "sara martin (Nickless) - Italy",
    "Ekaterina Nuzhdina": "Ekaterina Nuzdina (Nickless) - Russia",
    "Ashford Zoë": "Zoe Ashford (Zo_Ashford) - UK",
    "Richard Demsyn-Jones": "Richard Jones (rdj) - Canada",
    "Stackhouse Shawn": "S S (Nickless) - Canada",
    "Kuber Karthik": "Karthik K (krthk) - Canada",
    "Pranav Kamesh Sivakumar": "Pranav Kamesh (pranavmanu) - India",
    "Zoran Tanasić": "Zoran Tanasic (zorant) - Serbia",
    "Swaroop Srinivasrao Guggilam": "swaroop guggilam (swaroop2011) - India",
    "Nicole Schädel": "Nicole Schaedel (Mira) - Germany",
    "Karel Štěrba": "Karel Sterba (chlien) - Czech Rep.",
    "Jana Hanzelková": "Jana Hanzelkova (Tydela) - Czech Rep.",
    "Stefano Forcolin": "stefano forcolin (sf2l) - Italy",
    "Veronika Koľveková": "Veronika Kolvekova (Nickless) - Slovakia",
    "Kosei Yoshimori": "kosei yoshimori (simorin) - Japan",
    "Tomáš Krejčí": "Tom Krejčí (tierra) - Czech Rep.",
    "Robert Beärda": "Robert Bearda (Nickless) - Netherlands",
    "Raul Kačírek": "Raul Kacirek (Caca) - Czech Rep.",
    "Michele Bongiorno": "michele bongiorno (wiseman89) - Italy",
    "Daniele Colnaghi": "daniele colnaghi (SAYAN) - Italy",
    "Miroslav Šalaga": "Miroslav Salaga (macko) - Slovakia",
    "Jiwon Seo": "jiwon seo (Nickless) - Korea, South",
    "Gabriela Jaselská": "Gabriela Jaselska (Nickless) - Slovakia",
    "Blanka Lehotská": "Blanka Lehotska (hypsugo) - Slovakia",
    "Ján Farkaš": "Jan Farkas (slovak) - Slovakia",
    "Paolo Tivano": "paolo tivano (flin68) - Italy",
    "Iliana-Eleftheria Gounalaki": "Iliana Gounalaki (gounil) - Greece",
    "Peter Hornák": "Peter Hornak (petho) - Slovakia",
    "Pierdante Lanzavecchia": "Pierdante LANZAVECCHIA (Pierdante) - Italy",
    "Martin Fundárek": "Martin Fundarek (Nickless) - Slovakia",
    "Hyuksun Kwon": "hyuk sun Kwon (dolgore63) - Korea, South",
    "Aleš Založnik": "Ales Zaloznik (Aleš) - Slovenia",
    "Heungchul (John) Park": "Heungchul John Park (puzzlerepublic) - Korea, South",
    "Ľuboš Hyžák": "Lubos Hyzak (Nickless) - Slovakia",
    "Sunseong Kwon": "sunseong kwon (bomulsum) - Korea, South",
    "Saejoon Jang": "Sae Joon Jang (Nickless) - Korea, South",
    "Park Heungchul": "Heungchul John Park (puzzlerepublic) - Korea, South",
    "Mestrovic Davor": "Mladen Mestrovic (mestar) - Croatia",
    "Jan Novotny": "Jan Novotný (KrtekHonza) - Czech Rep.",
    "Klara Vytiskova": "Klára Vytisková (QKV) - Czech Rep.",
    "Hatice Esra Aydemir": "esra aydemir (aras) - Turkey",
    "Natalia Chanova": "Natália Chanová (natalka) - Slovakia",
    "Pavel Jaselsky": "Pavel Jaselský (pali7) - Slovakia",
    "Jana Vodickova": "Jana Vodičková (Janka) - Czech Rep.",
    "Hemant Kumar Malani": "Hemant Kr Malani (Nickless) - India",
    "Ralph Joshua P. Sarrosa": "Ralph Joshua Sarrosa (Nickless) - Philippines",
    "Cedomir Milanovic": "cedomir milanovic (rimodech) - Serbia",
    "Yuki Yamamoto": "Yuuki Yamamoto (brend) - Japan",
    "Deyan Razsadov ": "Deyan Razsadov (Deyan) - Bulgaria",
    "Svilen Dyakovski ": "Svilen Dyakovski (sdyakovski) - Bulgaria",
    "Charlotte Kroll": "Charlotte Anna Friederike Kroll (char.krol) - Germany",
    "John Joseph Dj Gabata": "John Joseph Gabata (Joseph) - Philippines",
    "Lenson Mithun Savio Andrade": "Lenson Andrade (lenson) - India",
    "Zdenek Vodicka": "Zdeněk Vodička (Voda) - Czech Rep.",
    "Aleksey Laptiev": "Alexey Laptiev (Kukuruza) - Russia",
    "Borislav Ilevski ": "Borislav Ilevski Ilevski (ilevski) - Bulgaria",
    "Arvid Jurgen Baars": "Arvid Baars (Eisbaer) - Netherlands",
    "Kulli Laks-Vahemae": "Külli Laks Vahemäe (Volvo) - Estonia",
    "Ondrousek Jakub": "Jakub Ondroušek (Nickless) - Czech Rep.",
    "David Tyler Jones": "David Jones (WMathie) - Canada",
    "Sridharan Kishore Kumar": "Kishore Sridharan (kishy72) - India",
    "Strozak Tomasz": "Tomasz Stróżak (strozo) - Poland",
    "Kadlecık Pavel": "Pavel Kadlečík (kousek-nebe) - Czech Rep.",
    "Gyimesi Zoltan": "Gyimesi Zoltán (Hunsudoku) - Hungary",
    "Meyapin Yannic": "Meyapin Yannick (Nickless) - France",
    "Zentgraf Jorg": "Jörg Zentgraf (Nickless) - Germany",
    "Mccaughan Emma": "Emma McCaughan (Emma) - UK",
    "Baines David": "Dave Baines (StandupCanada) - Canada",
    "Seung-Jae Kwak": "Seungjae Kwak (Kwaka) - Korea, South",
    "Jason Zuffraneiri": "Jason Zuffranieri (Ziti) - USA",
    "Štefan Gašpár": "Stefan Gaspar (pista) - Slovakia",
    "Gabriel Gan Rong De": "Gabriel Gan (rongde96) - Singapore",
    "Diana Škrhová": "Diana Skrhova (Anaid) - Slovakia",
    "Čedomir Milanović": "cedomir milanovic (rimodech) - Serbia",
    "Michael Moßhammer": "Michael Mosshammer (moss) - Austria",
    "Serkan Yürekli": "Serkan Yurekli (Nickless) - Turkey",
    "René Gilhuijs": "Rene Gilhuijs (RCG) - Netherlands",
    "Hugo Van Rooijen": "Hugo van Rooijen (Nickless) - Netherlands",
    "Heung-Chul(John) Park": "Heungchul John Park (puzzlerepublic) - Korea, South",
    "Ivana Štiptová": "Ivana Stiptova (tilansia) - Slovakia",
    "Frédéric Prevot": "Frédéric PREVOT (Ours Blanc) - France",
    "Swaroop Guggilam": "swaroop guggilam (swaroop2011) - India",
    "Veronika Macku": "Veronika Macků (Pherenike) - Czech Rep.",
    "Raoul Kačírek": "Raul Kacirek (Caca) - Czech Rep.",
    "Olga Díaz Moruno": "Olga Diaz (Nickless) - Spain",
    "Jouni Juhani Särkijärvi": "Jouni Särkijärvi (peluri) - Finland",
    "Beri Kohen Behar": "Beri Kohen (BERI) - Turkey",
    "Iliana-Eleytheria Gounalaki": "Iliana Gounalaki (gounil) - Greece",
    "Borislav Ilevski": "Borislav Ilevski Ilevski (ilevski) - Bulgaria",
    "Galina Titova": "Galya Titova (galka) - Bulgaria",
    "John Hc Park": "John HC Park (puzzlerepublic) - Korea, South",
    "Nikola Živanovic": "Nikola Zivanovic (NikolaZ) - Serbia",
    "Jana Brízová": "Jana Břízová (tojejedno) - Czech Rep.",
    "Fred Stalder": "Frédéric Stalder (Fred76) - Switzerland",
    "Pavel Kadlecík": "Pavel Kadlečík (kousek-nebe) - Czech Rep.",
    "Petra Cicová": "Petra Čičová (Nickless) - Czech Rep.",
    "Pawel Rachel": "Paweł Rachel (rachelinator) - Poland",
    "Jirí Hrdina": "Jiri Hrdina (jhrdina) - Czech Rep.",
    "Tomasz Strózak": "Tomasz Stróżak (strozo) - Poland",
    "Kerstin Woege": "Kerstin Wöge (Nickless) - Germany",
    "Joerg Zentgraf": "Jörg Zentgraf (Nickless) - Germany",
    "Laurent Saillot": "Laurent SAILLOT (LauLot57) - France",
    "Nikola Živanović": "Nikola Zivanovic (NikolaZ) - Serbia",
    "Wilbert Zwart": "Wilbert Zwart Wilbert (Space_wuppie) - Netherlands",
    "Zoran Таnasić": "Zoran Tanasic (zorant) - Serbia",
    "Jiří Hrdina": "Jiri Hrdina (jhrdina) - Czech Rep.",
    "Gülce Senem Özkütük Yürekli": "Gülçe Özkütük (Nickless) - Turkey",
    "Paweł Kępczyński": "Pawel Kepczynski (pafcio) - Poland",
    "Łukasz Kalinowski": "Lukasz Kalinowski (Nickless) - Poland",
    "Christiana Karakigianou": "Kristiana Karakigianou (farmatwnzwwn) - Greece",
    "Daisuke Takei": "Daisuke TAKEI (hotondo) - Japan",
    "Štefan Gyürki": "Stefan Gyürki (gypista) - Slovakia",
    "William Blatt": "Will Blatt (willwc) - USA",
    "Henning Poulsen": "Henning Kalsgaard Poulsen (Kalsgaard) - Denmark",
    "Tammy Mcleod": "Tammy McLeod (Nickless) - USA",
    "Recep Gül": "Recep GÜL (Nickless) - Turkey",
    "Mike Aisen": "Michael Aisen (maisen) - Canada",
    "Jan Zvérina": "Jan Zverina (Nickless) - Czech Rep.",
    "Jana Novotna": "Jana Novotná (tojejedno) - Czech Rep.",
    "Pavel Kadlecik": "Pavel Kadlečík (kousek-nebe) - Czech Rep.",
    "Andrej Plastiak": "Andrej Plaštiak (Nickless) - Slovakia",
    "Petra Cigova": "Petra Čičová (Nickless) - Czech Rep.",
    "Kerstin Wége": "Kerstin Wöge (Nickless) - Germany",
    "Gulia Franceschini": "Giulia Franceschini (iris) - Italy",
    "Przemystaw Debiak": "Przemysław Dębiak (Psyho) - Poland",
    "Karel Stérba": "Karel Sterba (chlien) - Czech Rep.",
    "Pranav Kamesh Sivak": "Pranav Kamesh (pranavmanu) - India",
    "Raul Ka¢Éirek": "Raul Kacirek (Caca) - Czech Rep.",
    "Akash Doulani": "akash doulani (akashdoulani78) - India",
    "Marek Kasar": "Marek Kasár (markas999) - Slovakia",
    "Maarit Ryynanen": "Maarit Ryynänen (Nickless) - Finland",
    "Ulla Elsila": "Ulla Elsilä (UllaE) - Finland",
    "Pawel Kepezynski": "Pawel Kepczynski (pafcio) - Poland",
    "Ashish Kumar": "ashish kumar (ashaash11ash) - India",
    "Gerda Nador": "Gerda Nádor (ingalili) - Hungary",
    "Gyérgy Herke": "György Herke (Gyuszi13) - Hungary",
    "Boglar Major": "Boglár Major (bmajor) - Hungary",
    "Fanni Pipé": "Fanni Pipó (Nickless) - Hungary",
    "Lucia Ondovéikovd": "Lucka Ondovcikova (Nickless) - Slovakia",
    "Jouni Sarkijarvi": "Jouni Särkijärvi (peluri) - Finland",
    "Tomasz Stanezak": "Tomasz Stańczak (stan) - Poland",
    "Claudio Toffon": "claudio toffon (claudio62) - Italy",
    "Evgenii Bekishev": "Evgeniy Bekishev (Eugene) - Russia",
    }

    wsc_and_gp["matched_id"] = wsc_and_gp.apply(
        lambda row: manual_map[row['flipped_name']] if row['flipped_name'] in manual_map else row['matched_id'], axis=1)
    wsc_and_gp["matched_id"] = wsc_and_gp.apply(
        lambda row: manual_map[row['Name']] if row['Name'] in manual_map else row['matched_id'], axis=1)
    
    wsc_and_gp["user_pseudo_id"] = wsc_and_gp["matched_id"]

    # TODO: Add in country
    wsc_and_gp["user_pseudo_id"] = wsc_and_gp["user_pseudo_id"].fillna(wsc_and_gp["Name"])

    return wsc_and_gp

wsc_matched_names = attemped_mapping(wsc, flattened)

no_matches = wsc_matched_names[wsc_matched_names["matched_id"].isna()]
print(f"No matches for {len(no_matches)} users of {len(wsc_matched_names)}")
display(no_matches.sort_values(by=["Official_rank"]).head(10))

No matches for 470 users of 2069


,year,Name,Official,Official_rank,Unofficial_rank,WSC_total,WSC_entry,WSC_t16 points,WSC_t1 points,WSC_t2 points,...,name_lc_y,Name_y,Country_y,Nick_y,user_pseudo_id_y,name_lc,Country,Nick,user_pseudo_id,matched_id
1954,2010,Chen Cen,True,7.0,7.0,1050,True,NaN,89.0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chen Cen,NaN
1649,2012,Chen Cen,True,7.0,7.0,396,True,NaN,33.0,109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chen Cen,NaN
1807,2011,Cen Chen,True,16.0,16.0,2827,True,NaN,52.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cen Chen,NaN
1964,2010,Ondřej Suchý,True,17.0,17.0,893,True,NaN,61.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ondřej Suchý,NaN
1967,2010,Gaurav Korde,True,20.0,20.0,871,True,NaN,72.0,79.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaurav Korde,NaN
1813,2011,Mehmet Murat Sevim,True,21.0,22.0,2622,True,NaN,72.0,230.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mehmet Murat Sevim,NaN
637,2018,Zuﬀranieri Jason,True,22.0,23.0,3265,True,NaN,215.0,420.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zuﬀranieri Jason,NaN
1667,2012,Frederique Rogeaux,True,25.0,25.0,294,True,NaN,31.0,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frederique Rogeaux,NaN
1974,2010,Akira Nakai,True,26.0,26.0,829,True,NaN,35.0,103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Akira Nakai,NaN
1818,2011,Frédérique Rogeaux,True,26.0,27.0,2586,True,NaN,76.0,185.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frédérique Rogeaux,NaN


In [4]:
import difflib

no_matches_subset = no_matches[no_matches["year"] == 2016]

# Function to find the closest match for each name in list1
def find_closest_match(list1, list2):
    closest_matches = {}
    for name in list1:
        closest = difflib.get_close_matches(name, list2, n=1, cutoff=0.0)
        closest_matches[name] = closest[0] if closest else None
    return closest_matches

# Get closest matches
matches = find_closest_match(no_matches_subset['Name'].astype(str), flattened["Name"])

for wsc_name in matches:
    gp_name = matches[wsc_name]
    gp_id = flattened[flattened["Name"] == gp_name]["user_pseudo_id"].iloc[0]
    print(f"\"{wsc_name}\": \"{gp_id}\",")

"Jan Zvérina": "Jan Zverina (Nickless) - Czech Rep.",
"Cen Chen": "IChun Chen (XPL0S1ON) - Taiwan",
"Jana Novotna": "Jana Novotná (tojejedno) - Czech Rep.",
"Pavel Kadlecik": "Pavel Kadlečík (kousek-nebe) - Czech Rep.",
"Andrej Plastiak": "Andrej Plaštiak (Nickless) - Slovakia",
"Pal Madarassy": "Paul McFreely (McFreely) - France",
"Olivier Rubio": "Olivier Lignon (Olivier Lignon) - France",
"Jim Schneider": "Tom Schneider (monkey61) - USA",
"Petra Cigova": "Petra Čičová (Nickless) - Czech Rep.",
"Kerstin Wége": "Kerstin Wöge (Nickless) - Germany",
"Gulia Franceschini": "Giulia Franceschini (iris) - Italy",
"Tamas Antal": "Tomasz Marcol (Nickless) - Poland",
"Przemystaw Debiak": "Przemysław Dębiak (Psyho) - Poland",
"Karel Stérba": "Karel Sterba (chlien) - Czech Rep.",
"Pranav Kamesh Sivak": "Pranav Kamesh (pranavmanu) - India",
"Raul Ka¢Éirek": "Raul Kacirek (Caca) - Czech Rep.",
"Niels Roest": "Miguel Dobles (TurtlesonFire) - Philippines",
"Akash Doulani": "akash doulani (akashdoulan

In [15]:
flattened[flattened["Name"].str.contains("lacoste")]

,user_pseudo_id,2014_1,2014_2,2014_3,2014_4,2014_5,2014_6,2014_7,2015_1,2015_2,...,2024_2,2024_3,2024_4,2024_5,2024_6,2024_7,2024_8,Name,Nick,Country


In [31]:
no_matches["year"].value_counts()

year
2018    81
2017    61
2010    57
2011    55
2012    50
2019    39
2015    39
2014    33
2023    27
2024    26
Name: count, dtype: int64